# ⚛️ H₃⁺ Noiseless VQE — Singles vs Doubles vs UCCSD

This notebook performs **noiseless VQE** for the trihydrogen cation **H₃⁺**, comparing:

- **UCC Singles**  
- **UCC Doubles**  
- **UCC Singles + Doubles (UCCSD)**  

For each excitation class, we:

- Build a corresponding VQE cost function  
- Optimize using **Adam**  
- Plot convergence  
- Reconstruct and print the final statevector in ket notation  
- (For UCCSD) draw the circuit  

This serves as the **baseline reference** for all future analyses:

- Noise scans  
- Mapping comparisons  
- Ansatz comparisons  

---

In [ ]:
# Core scientific stack
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem
import matplotlib.pyplot as plt
import sys, os, json

# Local imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
from vqe.vqe_utils import (
    IMG_DIR,
    excitation_ansatz,
    get_optimizer,
    set_seed,
    make_run_config_dict,
    run_signature,
    find_existing_run,
    save_run_record,
    ensure_dirs,
    build_run_filename,
)

ensure_dirs()

# 🔧 Molecular System (H₃⁺)

Geometry: equilateral triangle approximation  
Basis: **STO-3G**  
Charge: **+1**  
Electrons: **2**

In [ ]:
set_seed(0)

symbols = ["H", "H", "H"]
coordinates = np.array([
    [ 0.000000,  1.000000, 0.000000],
    [-0.866025, -0.500000, 0.000000],
    [ 0.866025, -0.500000, 0.000000],
])
basis = "STO-3G"
charge = +1
electrons = 2

hamiltonian, qubits = qchem.molecular_hamiltonian(
    symbols, coordinates, charge=charge, basis=basis
)

hf = qchem.hf_state(electrons, qubits)
num_wires = qubits

print(f"Qubits required: {qubits}")
print("HF state =", "|" + "".join(str(int(x)) for x in hf) + "⟩")

# 🔎 Excitations (Singles, Doubles)

We retrieve:

- **Singles**: 1 electron excitation  
- **Doubles**: 2-electron pair excitations  

In [ ]:
from pennylane.qchem import excitations

singles_raw, doubles_raw = excitations(electrons, qubits)

# Normalize to tuple form
singles = [tuple(x) for x in singles_raw]
doubles = [tuple(x) for x in doubles_raw]

print("Singles:", singles)
print("Doubles:", doubles)

# 🧩 Part A — Build VQE Cost Functions

A helper builds the correct ansatz for:

- `"single"` → UCC Singles  
- `"double"` → UCC Doubles  
- `"both"` → UCCSD (Singles + Doubles)  

The returned QNode always measures:

\[
E(\theta) = \langle \psi(\theta) | H | \psi(\theta) \rangle.
\]

In [ ]:
dev = qml.device("default.qubit", wires=num_wires)

def make_cost_fn(excitation_type):
    """Return a QNode computing the VQE energy for a given excitation class."""
    @qml.qnode(dev)
    def cost_fn(params):

        if excitation_type == "single":
            selected = (singles, [])
        elif excitation_type == "double":
            selected = ([], doubles)
        else:  # "both"
            selected = (singles, doubles)

        excitation_ansatz(
            params,
            wires=range(num_wires),
            hf_state=hf,
            excitations=selected,
            excitation_type=excitation_type,
        )
        return qml.expval(hamiltonian)

    return cost_fn

# 🧮 Part B — Cache-Aware VQE Runner

Handles:

- Reproducible configuration hashing  
- Cache lookup  
- Fresh computation if needed  
- JSON record creation  

In [ ]:
def run_vqe_for(excitation_type, stepsize, max_iterations):
    """Run (or load) VQE for singles / doubles / UCCSD."""
    if excitation_type == "single":
        n_params = len(singles)
        desc = "UCC Singles"
    elif excitation_type == "double":
        n_params = len(doubles)
        desc = "UCC Doubles"
    else:
        n_params = len(singles) + len(doubles)
        desc = "UCCSD"

    cost_fn = make_cost_fn(excitation_type)

    cfg = make_run_config_dict(
        symbols=symbols,
        coordinates=coordinates,
        basis=basis,
        ansatz_desc=desc,
        optimizer_name="Adam",
        stepsize=stepsize,
        max_iterations=max_iterations,
        seed=0,
    )
    cfg["excitation_type"] = excitation_type  # explicit

    sig = run_signature(cfg)
    fname = build_run_filename(f"H3plus_Noiseless_{excitation_type}", "Adam", 0, sig)
    existing = find_existing_run(sig)

    # Cached run
    if existing:
        print(f"[reuse] {excitation_type}: {existing}")
        with open(existing) as f:
            rec = json.load(f)
        return {
            "energies": rec["energies"],
            "params": np.array(rec["final_params"])
        }

    # Fresh run
    params = np.zeros(n_params, requires_grad=True)
    opt = get_optimizer("Adam", stepsize)

    energies = [cost_fn(params)]
    for _ in range(max_iterations):
        params, e_prev = opt.step_and_cost(cost_fn, params)
        energies.append(e_prev)

    # recompute final point for accuracy
    energies[-1] = float(cost_fn(params))

    record = {
        "config": cfg,
        "energies": [float(x) for x in energies],
        "final_params": [float(x) for x in params],
        "timestamp": __import__("time").strftime("%Y-%m-%d %H:%M:%S"),
    }
    save_run_record(fname, record)

    return {"energies": energies, "params": params}

# 🚀 Part C — Run VQE (Singles → Doubles → UCCSD)

In [ ]:
stepsize = 0.2
max_iterations = 50

excitation_types = ["single", "double", "both"]
results = {}

for exc in excitation_types:
    print(f"\n> Running VQE for: {exc}")
    results[exc] = run_vqe_for(exc, stepsize, max_iterations)

# 📉 Part D — Convergence Comparison

In [ ]:
plt.figure(figsize=(10, 6))

for exc in excitation_types:
    plt.plot(results[exc]["energies"], label=exc.capitalize())

plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.title("H₃⁺ VQE — Singles vs Doubles vs UCCSD")
plt.legend()
plt.grid(True, alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(IMG_DIR, "H3plus_Excitation_Comparison.png"), dpi=300)
plt.show()

# 🔬 Part E — Final Statevector Reconstruction

For each excitation type:

- Rebuild the final state  
- Extract amplitudes above a threshold  
- Format into readable ket notation  
- Draw the UCCSD circuit  

This gives full insight into the structure of the optimized state.

In [ ]:
def make_state_qnode(excitation_type, selected_excitations):
    """Return a QNode that outputs the full statevector."""
    @qml.qnode(dev)
    def circuit(params):
        excitation_ansatz(
            params,
            wires=range(num_wires),
            hf_state=hf,
            excitations=selected_excitations,
            excitation_type=excitation_type,
        )
        return qml.state()
    return circuit


threshold = 1e-2

for exc in excitation_types:
    print("\n" + "-"*80)
    print(f"State decomposition for: {exc.upper()}")

    params = results[exc]["params"]

    # Select excitations
    if exc == "single":
        selected = (singles, [])
    elif exc == "double":
        selected = ([], doubles)
    else:
        selected = (singles, doubles)

    state_fn = make_state_qnode(exc, selected)
    psi = state_fn(params)

    # Extract significant amplitudes
    nz = np.where(np.abs(psi) > threshold)[0]

    ket_terms = []
    for idx in nz:
        amp = psi[idx]
        bit = f"|{idx:0{num_wires}b}⟩"
        amp_str = f"{amp.real:.4f}" if abs(amp.imag) < 1e-6 else f"({amp.real:.4f} + {amp.imag:.4f}j)"
        ket_terms.append(f"{amp_str}{bit}")

    print(" |ψ⟩ =", " + ".join(ket_terms))

    # Draw circuit only for UCCSD
    if exc == "both":
        fig, ax = qml.draw_mpl(state_fn)(params)
        fig.suptitle("H₃⁺ — UCCSD Circuit", fontsize=15)
        fig.savefig(os.path.join(IMG_DIR, "H3plus_UCCSD_Circuit.png"), dpi=300)
        plt.show()